# Creación Dimensión Clientes

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, current_timestamp

# 1. Leer desde Silver y asegurar registros únicos
df_silver_cli = spark.read.table("workspace.silver.clientes").dropDuplicates(["numero_identificacion"])

# 2. Definir ventana para la Clave Subrogada
window_spec_cli = Window.orderBy("numero_identificacion")

# 3. Crear la Dimensión Gold
dim_clientes = df_silver_cli.withColumn("cliente_key", row_number().over(window_spec_cli)) \
    .select(
        col("cliente_key"), # SK
        col("numero_identificacion").alias("identificacion_bk"), # BK
        col("tipo_identificacion"),
        col("razon_social"),
        col("nombre_comercial"),
        col("direccion"),
        col("telefono"),
        col("email"),
        col("grupo"),
        current_timestamp().alias("fecha_carga")
    )

# 4. Guardar en Gold
dim_clientes.write.mode("overwrite").saveAsTable("workspace.gold.dim_clientes")

print("✅ dim_clientes creada exitosamente en Capa Gold.")